In [ ]:
import streamlit as st
import google.generativeai as genai
import speech_recognition as sr
import pyttsx3
import os
from dotenv import load_dotenv
import threading
import time

# Load environment variables
load_dotenv()

# Configure API key
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
genai.configure(api_key=GOOGLE_API_KEY)

# Set up the model
model = genai.GenerativeModel(
    model_name="gemini-1.5-flash",
    generation_config={
        "temperature": 0.7,
        "top_p": 0.95,
        "top_k": 40,
        "max_output_tokens": 1024,
    }
)

# Initialize chat history
if "chat_history" not in st.session_state:
    st.session_state.chat_history = []
    
if "conversation" not in st.session_state:
    st.session_state.conversation = model.start_chat(
        history=[],
    )


In [ ]:
# Initialize the speech engine
engine = pyttsx3.init()
voices = engine.getProperty('voices')
engine.setProperty('voice', voices[1].id)  # Use female voice
engine.setProperty('rate', 175)  # Speed of speech

# Speech recognition setup
recognizer = sr.Recognizer()
recognizer.energy_threshold = 4000
recognizer.dynamic_energy_threshold = True

def speak_text(text):
    """Function to convert text to speech"""
    engine.say(text)
    engine.runAndWait()


In [ ]:
def listen_for_speech():
    """Function to capture speech from microphone and convert to text"""
    with sr.Microphone() as source:
        st.info("Listening... Speak now.")
        recognizer.adjust_for_ambient_noise(source, duration=1)
        try:
            audio = recognizer.listen(source, timeout=5, phrase_time_limit=15)
            st.info("Processing speech...")
            text = recognizer.recognize_google(audio)
            return text
        except sr.WaitTimeoutError:
            return "No speech detected."
        except sr.UnknownValueError:
            return "Sorry, I couldn't understand what you said."
        except sr.RequestError:
            return "Speech service is unavailable."
        except Exception as e:
            return f"Error: {str(e)}"

In [ ]:
def get_ai_response(user_input):
    """Get response from Gemini model"""
    response = st.session_state.conversation.send_message(user_input)
    return response.text

In [ ]:
def speak_response_threaded(text):
    """Speak the response in a separate thread to not block the UI"""
    thread = threading.Thread(target=speak_text, args=(text,))
    thread.start()

In [ ]:
# Streamlit UI
st.title("🤖 Voice AI Companion")
st.subheader("Powered by Gemini 1.5 Flash")

# Display chat history
chat_container = st.container()
with chat_container:
    for message in st.session_state.chat_history:
        if message["role"] == "user":
            st.markdown(f"**You:** {message['content']}")
        else:
            st.markdown(f"**AI:** {message['content']}")

# Voice input button
col1, col2 = st.columns(2)

with col1:
    if st.button("🎤 Speak to AI"):
        user_input = listen_for_speech()
        if user_input and user_input not in ["No speech detected.", "Sorry, I couldn't understand what you said.", "Speech service is unavailable."]:
            # Add user message to chat history
            st.session_state.chat_history.append({"role": "user", "content": user_input})
            
            # Get AI response
            with st.spinner("AI is thinking..."):
                ai_response = get_ai_response(user_input)
                
            # Add AI response to chat history
            st.session_state.chat_history.append({"role": "assistant", "content": ai_response})
            
            # Speak the response
            speak_response_threaded(ai_response)
            
            # Force refresh to show new messages
            st.experimental_rerun()
        else:
            st.warning(user_input)


In [ ]:
# Text input as an alternative to voice
with col2:
    user_input = st.text_input("Or type your message:", key="text_input")
    if st.button("Send"):
        if user_input:
            # Add user message to chat history
            st.session_state.chat_history.append({"role": "user", "content": user_input})
            
            # Get AI response
            with st.spinner("AI is thinking..."):
                ai_response = get_ai_response(user_input)
                
            # Add AI response to chat history
            st.session_state.chat_history.append({"role": "assistant", "content": ai_response})
            
            # Speak the response
            speak_response_threaded(ai_response)
            
            # Clear the text input
            st.session_state.text_input = ""
            
            # Force refresh to show new messages
            st.experimental_rerun()

# Reset conversation button
if st.button("Reset Conversation"):
    st.session_state.chat_history = []
    st.session_state.conversation = model.start_chat(history=[])
    st.success("Conversation has been reset!")
    time.sleep(1)
    st.experimental_rerun()

In [6]:
import streamlit as st
import google.generativeai as genai
import speech_recognition as sr
import pyttsx3
import os
from dotenv import load_dotenv
import threading
import time

# Load environment variables
load_dotenv()

# Verify API key loading
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
if GOOGLE_API_KEY:
    st.sidebar.success("API key loaded successfully")
    print(f"API key found: {GOOGLE_API_KEY[:4]}...")  # Print first 4 chars for safety
else:
    st.sidebar.error("API key not found in .env file")
    print("API key not found")

# Configure API key
genai.configure(api_key=GOOGLE_API_KEY)

# Rest of the code remains the same

NameError: name 'client' is not defined